# Pendientes

* convertir control en diccionario
* script mas complejo para proporcion
    - puede haber nulos, solucionar los faltantes
    - poder darle una lista de meses
* arreglar scripts para control
*reescribir optimizaicon de hiperparam

In [26]:
from datetime import datetime
import os

In [27]:
NOMBRE_EXPERIMENTO = '2024-06-23_NuevoScript'

FILES = [
    ['01_sell_in_group.parquet',
        '01_prod_stats.parquet',
        '01_prod_data_group.parquet',
        '01_prod_data_ungroup.parquet',
        '01_stock_data.parquet'],
    ['02_sell_in_norm.parquet'],
    ['03_sell_in_lag.parquet'],
    ['04_sell_in_train.parquet',
        '04_sell_in_test.parquet',
        '04_sell_in_futuro.parquet'],
    ['05_sell_in_pred.parquet',
        '05_sell_in_pred_futuro.parquet'],
    ['06_kaggle_pred.csv'],
]

FOLDER = f'./Experimentos/{NOMBRE_EXPERIMENTO}/'

In [28]:
for FILE in FILES:
    for f in FILE:
        print(f"{FOLDER}{f}")

./Experimentos/2024-06-23_NuevoScript/01_sell_in_group.parquet
./Experimentos/2024-06-23_NuevoScript/01_prod_stats.parquet
./Experimentos/2024-06-23_NuevoScript/01_prod_data_group.parquet
./Experimentos/2024-06-23_NuevoScript/01_prod_data_ungroup.parquet
./Experimentos/2024-06-23_NuevoScript/01_stock_data.parquet
./Experimentos/2024-06-23_NuevoScript/02_sell_in_norm.parquet
./Experimentos/2024-06-23_NuevoScript/03_sell_in_lag.parquet
./Experimentos/2024-06-23_NuevoScript/04_sell_in_train.parquet
./Experimentos/2024-06-23_NuevoScript/04_sell_in_test.parquet
./Experimentos/2024-06-23_NuevoScript/04_sell_in_futuro.parquet
./Experimentos/2024-06-23_NuevoScript/05_sell_in_pred.parquet
./Experimentos/2024-06-23_NuevoScript/05_sell_in_pred_futuro.parquet
./Experimentos/2024-06-23_NuevoScript/06_kaggle_pred.csv


In [29]:
%run 01_LecturaDatos.ipynb
%run 02_normaliza.ipynb
%run 03_features.ipynb
%run 04_crear_train_set.ipynb
%run 05_lightgbm.ipynb
%run 06_evaluar_prediccion.ipynb

------------------------------COMIENZA-----------------------------
[./Experimentos/2024-06-23_NuevoScript//['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data_group.parquet', '01_prod_data_ungroup.parquet', '01_stock_data.parquet']
(15416, 3)
(542, 11)
 ./Experimentos/2024-06-23_NuevoScript//['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data_group.parquet', '01_prod_data_ungroup.parquet', '01_stock_data.parquet']
------------------------------FINALIZA-----------------------------
------------------------------COMIENZA-----------------------------
[./Experimentos/2024-06-23_NuevoScript//['02_sell_in_norm.parquet']
 ./Experimentos/2024-06-23_NuevoScript//['02_sell_in_norm.parquet']
------------------------------FINALIZA-----------------------------
------------------------------COMIENZA-----------------------------
[./Experimentos/2024-06-23_NuevoScript//['03_sell_in_lag.parquet']
(542, 20)
(542, 168)
 ./Experimentos/2024-06-23_NuevoScript//['03_sell_in_

In [30]:
# %run 01_LecturaDatos.ipynb
# %run 02_normaliza.ipynb
# %run 03_features.ipynb
# %run 04_crear_train_set.ipynb
# %run 05_lightgbm.ipynb
# %run 06_evaluar_prediccion.ipynb